In [ ]:
%matplotlib notebook
import dill
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [ ]:

filename = '/Users/bpb/Downloads/positive_mode_isotope_data.pkl'
with open(filename,'rb') as fid:
    data = dill.load(fid)

In [ ]:
for i,d in enumerate(data):
    data[i]['rt_peak'] = pd.DataFrame(data[i]['rt_peak'])
    data[i]['peak_height'] = pd.DataFrame(data[i]['peak_height'])

In [ ]:
[c.replace('20171221_TS_DB_fullcyc_QE144_50454-738379_','') for c in data[0]['rt_peak'].columns.tolist()]

In [ ]:
[(i,d['compound']) for i,d in enumerate(data)]

In [ ]:
cpd_id = 30

In [ ]:
len(data)

In [ ]:
data[cpd_id].keys()

In [ ]:
data[cpd_id]['compound']

In [ ]:
data[cpd_id]['rt_peak'].columns[:3]

In [ ]:
data[cpd_id]['peak_height'].columns[:3]

In [ ]:
treatments = list(set(['_'.join(c.split('_')[7:9]) for c in data[cpd_id]['peak_height'].columns if 'D' in c]))
treatments

In [ ]:
time_points = []
for c in data[cpd_id]['peak_height'].columns:
    try:
        found = re.search('_D\d+_', c).group()
        time_points.append(found.strip('_'))
    except AttributeError:
        pass
time_points = list(set(time_points))
time_points

In [ ]:
groups = list(set(['_'.join(c.split('_')[7:10]) for c in data[cpd_id]['peak_height'].columns if 'D48' in c]))
groups

In [ ]:
peak_height = data[cpd_id]['peak_height']
control = peak_height[[c for c in peak_height.columns if groups[0] in c]].mean(axis=1)
control

In [ ]:
labeled = peak_height[[c for c in peak_height.columns if groups[2] in c]].mean(axis=1)
labeled

In [ ]:
labeled - control

In [ ]:
diff_data = {}
for g in groups:
    sub_data = peak_height[[c for c in peak_height.columns if g in c]].mean(axis=1)
    diff_data[g] = sub_data
df = pd.DataFrame(diff_data)
df

In [ ]:
plt.ioff()
for time_point in time_points:
    folder_name = 'isotope_figs'
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    for cpd_id in range(len(data)):
        peak_height = data[cpd_id]['peak_height']
        diff_data = {}
        fig = plt.figure()
        ax = fig.gca()
        for treatment in treatments:
            g = '%s_%s'%(treatment,time_point)
            sub_data = peak_height[[c for c in peak_height.columns if g in c]].mean(axis=1)
            diff_data[g] = sub_data
        df = pd.DataFrame(diff_data)
        df.index = [d.replace('p','.') for d in df.index]
        df[:-1].plot(kind='bar',ax=ax) #drop the unnecessary last ion
        plt.title(data[cpd_id]['compound'],fontsize=18)
        ax.set_yscale('log')
        plt.ylabel('Peak Height (au)')
        plt.xlabel('m/z')
        plt.tight_layout()
        fig.savefig(os.path.join(folder_name,'%s_%s.pdf'%(data[cpd_id]['compound'],time_point)))
        fig.clear()
plt.ion()